## Quantitative trading in China A stock market with FinRL

<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/tutorials/3-Practical/FinRL_China_A_Share_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install FinRL

In [1]:
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-ciy2m3h0
  Running command git clone -q https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-ciy2m3h0
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-4o27_l2q/pyfolio_6efbd029893f4cb79efa61dacde0929b
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-4o27_l2q/pyfolio_6efbd029893f4cb79efa61dacde0929b
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-4o27_l2q/elegantrl_1071d05d9be8433f8be24674aa800ddd
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-4o27_l2q/elegantrl_1071d05d9be8433f8be24674aa800ddd
     |████████████████████████████████| 2.3 MB 7.6 MB/s 
     |████████████████████████████████| 234 kB 54.5 MB/s 
     |████████████████████████

Install other libraries

In [2]:
# !pip install stockstats
# !pip install tushare
# #install talib
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') 
# !./configure --prefix=/usr
# !make
# !make install
# os.chdir('../')
# !pip install TA-Lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 45.2 MB/s 
     |████████████████████████████████| 200 kB 60.5 MB/s 
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.3.3
    Uninstalling websocket-client-1.3.3:
      Successfully uninstalled websocket-client-1.3.3
--2022-08-11 07:28:50--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.g

In [3]:
# %cd /
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
# %cd /FinRL-Meta/

/
Cloning into 'FinRL-Meta'...
remote: Enumerating objects: 7096, done.
remote: Counting objects: 100% (780/780), done.
remote: Compressing objects: 100% (510/510), done.
remote: Total 7096 (delta 346), reused 555 (delta 257), pack-reused 6316
Receiving objects: 100% (7096/7096), 137.37 MiB | 26.20 MiB/s, done.
Resolving deltas: 100% (3817/3817), done.
Checking out files: 100% (442/442), done.
/FinRL-Meta


### Import modules

In [52]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
from IPython import display

display.set_matplotlib_formats("svg")

from agents.stablebaselines3_models import DRLAgent
from meta import config
from meta.data_processors.tushare_private import ReturnPlotterPrivate, TusharePrivate
from meta.env_stock_trading.env_stocktrading_China_A_shares import \
    StockTradingEnv

pd.options.display.max_columns = None
    
print("ALL Modules have been imported!")

ALL Modules have been imported!


### Create folders

In [53]:
import os

if not os.path.exists("./datasets" ):
    os.makedirs("./datasets" )
if not os.path.exists("./trained_models"):
    os.makedirs("./trained_models" )
if not os.path.exists("./tensorboard_log"):
    os.makedirs("./tensorboard_log" )
if not os.path.exists("./results" ):
    os.makedirs("./results" )

### Download data, cleaning and feature engineering

In [54]:
# import importlib

# import meta
# from meta.local.selector.stock_selector import StockSelector

# importlib.reload(meta.local.selector.stock_selector)

# stock_sel = StockSelector()
# ticket_list = stock_sel.select_stocks()
# print(len(ticket_list))

# train_start_date='2015-01-01'
# train_stop_date='2019-08-01'
# val_start_date='2019-08-01'
# val_stop_date='2022-10-27'

# token='27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'


ticket_list=['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH',
       '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH',
       '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

train_start_date='2015-01-01'
train_stop_date='2019-08-01'
val_start_date='2019-08-01'
val_stop_date='2022-10-27'

token='27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'

In [55]:
# download and clean
ts_processor = TusharePrivate(data_source="tushare", 
                                   start_date=train_start_date,
                                   end_date=val_stop_date,
                                   time_interval="1d",
                                   token=token)
ts_processor.download_data(ticker_list=ticket_list)

100%|██████████| 15/15 [00:10<00:00,  1.37it/s]

Shape of DataFrame:  (28142, 8)


In [56]:
ts_processor.clean_data()
ts_processor.dataframe

Shape of DataFrame:  (28515, 8)


,tic,date,open,high,low,close,volume,day
0,600000.SH,2015-01-05,15.88,16.25,15.56,16.07,5135687.09,0.0
1,600009.SH,2015-01-05,19.82,20.91,19.82,20.53,371485.54,0.0
2,600016.SH,2015-01-05,10.87,10.96,10.50,10.78,9138873.70,0.0
3,600028.SH,2015-01-05,6.59,7.14,6.45,7.14,11864996.45,0.0
4,600030.SH,2015-01-05,33.90,35.25,33.01,34.66,6986272.15,0.0
...,...,...,...,...,...,...,...,...
28510,600276.SH,2022-10-27,39.72,40.15,39.20,39.72,507585.76,3.0
28511,600309.SH,2022-10-27,82.08,84.15,82.08,83.13,121248.37,3.0
28512,600519.SH,2022-10-27,1480.00,1487.98,1383.85,1401.00,107105.90,3.0
28513,600547.SH,2022-10-27,17.28,17.66,17.25,17.55,246851.79,3.0


In [57]:
# add_technical_indicator
ts_processor.add_technical_indicator(config.INDICATORS)
ts_processor.add_turbulence()
ts_processor.clean_data()
ts_processor.dataframe

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (28470, 18)


,tic,date,index,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,3306271.72,3.0,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.815000,15.815000,0.000000
1,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,198117.45,3.0,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.200000,20.200000,0.000000
2,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,4851684.17,3.0,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.477500,10.477500,0.000000
3,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,8190902.35,3.0,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.042500,7.042500,0.000000
4,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,6376268.69,3.0,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.192500,35.192500,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28465,600276.SH,2022-10-27,28510,39.72,40.15,39.20,39.72,507585.76,3.0,1.348563,42.223050,30.762950,58.848811,119.283962,26.043118,35.826000,35.663167,28.612419
28466,600309.SH,2022-10-27,28511,82.08,84.15,82.08,83.13,121248.37,3.0,-1.486713,95.717280,81.371720,43.454133,-156.438783,34.119441,89.388333,87.700000,28.612419
28467,600519.SH,2022-10-27,28512,1480.00,1487.98,1383.85,1401.00,107105.90,3.0,-102.711452,2012.276307,1404.804693,24.340408,-201.425183,75.222076,1757.868333,1823.905333,28.612419
28468,600547.SH,2022-10-27,28513,17.28,17.66,17.25,17.55,246851.79,3.0,-0.115207,17.616401,16.578599,49.339713,19.282091,11.622283,17.365667,17.765500,28.612419


### Split traning dataset

In [58]:
train =ts_processor.data_split(ts_processor.dataframe, train_start_date, train_stop_date)       
len(train.tic.unique())

15

In [59]:
train.tic.unique()

array(['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH',
       '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH',
       '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH'],
      dtype=object)

In [60]:
train.head()

,tic,date,index,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,3306271.72,3.0,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150,0.0
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,198117.45,3.0,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000,0.0
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,4851684.17,3.0,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775,0.0
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,8190902.35,3.0,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425,0.0
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,6376268.69,3.0,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925,0.0


In [61]:
train.shape

(16695, 18)

In [62]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension*(len(config.INDICATORS)+2)+1
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


### Train

In [63]:
env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct":6.87e-5,
    "sell_cost_pct":1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space, 
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS, 
    "print_verbosity": 1,
    "initial_buy":True,
    "hundred_each_trade":True
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## DDPG

In [64]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [65]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {
                "batch_size": 256, 
               "buffer_size": 50000, 
               "learning_rate": 0.05,
               "action_noise":"normal",
                }
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300]))
model_ddpg = agent.get_model("ddpg", model_kwargs = DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.05, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device


In [79]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                              tb_log_name='ddpg',
                              total_timesteps=100000,
                              n_eval_episodes=50)


Logging to tensorboard_log/ddpg/ddpg_10
Episode: 12
day: 1112, episode: 12
begin_total_asset: 1000000.00
end_total_asset: 2235266.42
total_reward: 1235266.42
total_cost: 12512.73
total_trades: 16678
Sharpe: 0.855
Episode: 13
day: 1112, episode: 13
begin_total_asset: 1000000.00
end_total_asset: 1748834.86
total_reward: 748834.86
total_cost: 456.14
total_trades: 16680
Sharpe: 0.609
Episode: 14
day: 1112, episode: 14
begin_total_asset: 1000000.00
end_total_asset: 1659595.84
total_reward: 659595.84
total_cost: 456.16
total_trades: 16680
Sharpe: 0.566
Episode: 15
day: 1112, episode: 15
begin_total_asset: 1000000.00
end_total_asset: 1668560.84
total_reward: 668560.84
total_cost: 456.16
total_trades: 16680
Sharpe: 0.569
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 68        |
|    time_elapsed    | 64        |
|    total_timesteps | 4452      |
| train/             |           |
|    actor_loss      | 17.4     

## A2C

In [3]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

NameError: name 'DRLAgent' is not defined

In [ ]:
# trained_a2c = agent.train_model(model=model_a2c, 
#                              tb_log_name='a2c',
#                              total_timesteps=50000)

### Trade

In [ ]:
trade = ts_processor.data_split(ts_processor.dataframe, val_start_date, val_stop_date)
env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct":6.87e-5,
    "sell_cost_pct":1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space, 
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS, 
    "print_verbosity": 1,
    "initial_buy":False,
    "hundred_each_trade":True
}
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg,
                       environment = e_trade_gym)

Episode: 2
day: 783, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 633232.33
total_reward: -366767.67
total_cost: 68.67
total_trades: 4698
Sharpe: -0.520
hit end!


In [ ]:
df_actions.to_csv("action.csv",index=False)
df_actions

,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600050.SH,600104.SH,600196.SH,600276.SH,600309.SH,600519.SH,600547.SH,600570.SH
date,,,,,,,,,,,,,,,
2019-08-01,0,1000,1000,1000,0,0,1000,0,1000,0,0,0,0,1000,0
2019-08-02,0,1000,1000,1000,0,0,1000,0,1000,0,0,0,0,1000,0
2019-08-05,0,1000,1000,1000,0,0,1000,0,1000,0,0,0,0,1000,0
2019-08-06,0,1000,1000,1000,0,0,1000,0,1000,0,0,0,0,1000,0
2019-08-07,0,1000,1000,1000,0,0,1000,0,1000,0,0,0,0,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022-10-20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022-10-21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Backtest

In [ ]:
# %matplotlib inline
plotter = ReturnPlotterPrivate(df_account_value, trade, val_start_date, val_stop_date)
# plotter.plot_all()

NameError: name 'ReturnPlotterPrivate' is not defined

In [ ]:
%matplotlib inline
plotter.plot(figure_filepath="./China_A_share.png")

NameError: name 'plotter' is not defined

In [ ]:
# %matplotlib inline
# # ticket: SSE 50：000016
# plotter.plot("000016")

#### Use pyfolio

In [ ]:
# CSI 300
baseline_df = plotter.get_baseline("600000.SH")    # "399300"

In [ ]:
import pyfolio
from pyfolio import timeseries
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return, 
                              factor_returns=daily_return_base, 
                                positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


Annual return         -0.136591
Cumulative returns    -0.366768
Annual volatility      0.231233
Sharpe ratio          -0.519682
Calmar ratio          -0.325949
Stability              0.818270
Max drawdown          -0.419055
Omega ratio            0.913559
Sortino ratio         -0.692743
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.888498
Daily value at risk   -0.029610
Alpha                       NaN
Beta                        NaN
dtype: float64

In [ ]:
import pyfolio
from pyfolio import timeseries
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, 
                              factor_returns=daily_return_base, 
                                positions=None, transactions=None, turnover_denom="AGB")
print("==============Baseline Strategy Stats===========")
perf_stats_all

==============Baseline Strategy Stats===========


Annual return          0.187546
Cumulative returns     0.708211
Annual volatility      0.195643
Sharpe ratio           0.977404
Calmar ratio           0.969515
Stability              0.860908
Max drawdown          -0.193443
Omega ratio            1.198114
Sortino ratio          1.494488
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.055923
Daily value at risk   -0.023890
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [ ]:
# with pyfolio.plotting.plotting_context(font_scale=1.1):
#         pyfolio.create_full_tear_sheet(returns = daily_return,
#                                        benchmark_rets = daily_return_base, set_context=False)

### Authors
github username: oliverwang15, eitin-infant